<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/outliers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Based on  https://www.kaggle.com/datasets/yasserh/uber-fares-dataset?resource=download
# https://careerfoundry.com/en/blog/data-analytics/how-to-find-outliers/

In [36]:
import pandas as pd
import numpy as np

In [37]:
datos_oscar = pd.Series([10, 20, 14, 11], 
            index=['a', 'b', 'c', 'd'])
datos_oscar
type(datos_oscar)

pandas.core.series.Series

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/armandoordonez/eda_couse/main/data/uber.csv')

In [5]:
# Eliminamos la coulumnas que no usamos

df = df.drop(columns=(['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']))

In [6]:
df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,5


In [7]:
# Observamos los altos valores en las columnas
df[['fare_amount', 'passenger_count']].describe()

,fare_amount,passenger_count
count,200000.000000,200000.000000
mean,11.359955,1.684535
std,9.901776,1.385997
min,-52.000000,0.000000
25%,6.000000,1.000000
50%,8.500000,1.000000
75%,12.500000,2.000000
max,499.000000,208.000000


In [8]:
df.describe()[['fare_amount', 'passenger_count']]

,fare_amount,passenger_count
count,200000.000000,200000.000000
mean,11.359955,1.684535
std,9.901776,1.385997
min,-52.000000,0.000000
25%,6.000000,1.000000
50%,8.500000,1.000000
75%,12.500000,2.000000
max,499.000000,208.000000


Como podemos ver, las columnas fare_amount y Passenger_Count tienen valores atípicos. Por ejemplo, el importe máximo de la tarifa es 499, mientras que la media es 11,36. La media es sensible a los valores atípicos, pero el hecho de que la media sea tan pequeña en comparación con el valor máximo indica que el valor máximo es un valor atípico. Del mismo modo, el número máximo de pasajeros es 208, mientras que la media es 1,68. Dado que el conductor ingresa este valor, mi mejor suposición para el valor atípico del número de pasajeros es un error humano. A medida que exploramos los datos utilizando métodos adicionales, podemos decidir cómo manejar los valores atípicos.

In [9]:
# Graficamos 

import plotly.express as px
fig = px.histogram(df, x='fare_amount')
fig.show()

# Box



In [10]:
fig = px.box(df, y='fare_amount')
fig.show()

In [11]:
# Grafico de dispersión
fig = px.scatter(x=df['passenger_count'], y=df['fare_amount'])
fig.show()

In [12]:
#Función para calcular IQR

def find_outliers_IQR(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   outliers = df[ ((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR))) ]
   return outliers

In [17]:
outliers = find_outliers_IQR(df['fare_amount'])

print('número de outliers: '+ str(len(outliers)))
print('max valor outlier: ' + str(outliers.max()))
print('min valor outlier: ' + str(outliers.min()))

print(outliers)

número de outliers: 17167
max valor outlier: 499.0
min valor outlier: -52.0
6         24.50
30        25.70
34        39.50
39        29.00
48        56.80
          ...  
199976    49.70
199977    43.50
199982    57.33
199985    24.00
199997    30.90
Name: fare_amount, Length: 17167, dtype: float64


In [18]:
# Crera una función para para encontrar outliers con  IQR

def drop_outliers_IQR(df):

   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   
   # No outliers
   not_outliers = df[~((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
   
   # datos limpios
   outliers_clean = not_outliers.dropna().reset_index()

   return outliers_clean

In [19]:
# LLamamos la función

outliers_clean = drop_outliers_IQR(df['fare_amount'])

print('number of outliers: '+ str(len(outliers_clean)))
print('max outlier value: ' + str(outliers_clean.max()))
print('min outlier value: ' + str(outliers_clean.min()))

print(outliers_clean)

number of outliers: 182833
max outlier value: index          199999.0
fare_amount        22.2
dtype: float64
min outlier value: index          0.0
fare_amount   -3.5
dtype: float64
         index  fare_amount
0            0          7.5
1            1          7.7
2            2         12.9
3            3          5.3
4            4         16.0
...        ...          ...
182828  199994         12.0
182829  199995          3.0
182830  199996          7.5
182831  199998         14.5
182832  199999         14.1

[182833 rows x 2 columns]


In [20]:
type(outliers)

pandas.core.series.Series

# Alternativa de colocar el valor máximo y mínimo

In [23]:
# Identificamos los valores máximo y mínimo 

upper_limit = df['fare_amount'].mean() + 3*df['fare_amount'].std()
lower_limit = df['fare_amount'].mean() - 3* df['fare_amount'].std()
print(lower_limit)
print(upper_limit)

-18.345373425209697
41.0652839252097


In [29]:
# Ejemplo del uso del uso de where
# https://www.google.com/search?q=np.where&sxsrf=AJOqlzUb20zzkapz12UGHXYmCU93RI_r-g:1678479409523&source=lnms&tbm=vid&sa=X&ved=2ahUKEwi1od6Hl9L9AhVgfTABHbx6AZ4Q_AUoAnoECAEQBA&biw=1364&bih=697&dpr=1
# Creamos un arreglo de 10 elementos
a = np.arange(10)
print(type(a))
print (a)

# si se cumple la condición "a<5" se coloca el valor de a (0,1,2,3,4) cuando no se cumple la condición se aplica la segunda formula 10*a (50,60,70,80,90)
b = np.where(a < 5, a, 10*a)
print(b)

<class 'numpy.ndarray'>
[0 1 2 3 4 5 6 7 8 9]
[ 0  1  2  3  4 50 60 70 80 90]


In [30]:
# cuando df['fare_amount'] > upper_limit colocar upper limit
# cuando df['fare_amount'] < lower_limit colocar lower_limit

df['fare_amount'] = np.where(df['fare_amount'] > upper_limit, upper_limit, np.where( df['fare_amount'] < lower_limit,lower_limit, df['fare_amount']))

In [31]:
df['fare_amount']

0          7.5
1          7.7
2         12.9
3          5.3
4         16.0
          ... 
199995     3.0
199996     7.5
199997    30.9
199998    14.5
199999    14.1
Name: fare_amount, Length: 200000, dtype: float64

In [32]:
df.describe()

,Unnamed: 0,fare_amount,passenger_count
count,2.000000e+05,200000.000000,200000.000000
mean,2.771250e+07,11.008988,1.684535
std,1.601382e+07,8.088084,1.385997
min,1.000000e+00,-18.345373,0.000000
25%,1.382535e+07,6.000000,1.000000
50%,2.774550e+07,8.500000,1.000000
75%,4.155530e+07,12.500000,2.000000
max,5.542357e+07,41.065284,208.000000


In [33]:
# Observamos el máximo y el mínimo

df.describe()[['fare_amount']]

,fare_amount
count,200000.000000
mean,11.008988
std,8.088084
min,-18.345373
25%,6.000000
50%,8.500000
75%,12.500000
max,41.065284


In [34]:
# Imputamos los valores por la media

def impute_outliers_IQR(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   upper = df[~(df>(q3+1.5*IQR))].max()
   lower = df[~(df<(q1-1.5*IQR))].min()

   df = np.where(df > upper, # donde sea mayor 
       df.mean(),             # colocamos la media
       np.where(              # en otros caso:
           df < lower,        # Cuando sea menor a lower
           df.mean(),         # colocamos la media
           df                 # el resto de las veces dejamos el valor como estaba
           )
       )
   return df

In [35]:
df['fare_amount'] = impute_outliers_IQR(df['fare_amount'])
df.describe()['fare_amount']

count    200000.000000
mean          9.117679
std           4.003623
min          -3.500000
25%           6.000000
50%           8.500000
75%          11.008988
max          22.200000
Name: fare_amount, dtype: float64

# Ejercicio

Realice la revisión de outliers a una de las columnas de su datase